
# Лабаораторная работа №1: Аудио EDA и характеристики сигнала

**Цель:** научиться загружать аудиодатасеты, читать и интерпретировать метаданные, вычислять базовые характеристики аудиосигнала и строить визуализации (waveform, спектр, Mel/Log‑Mel).  
**Формат:** заполнить `TODO`‑ячейки. Ячейки с `raise NotImplementedError` необходимо заменить вашим кодом.

**Требования по оформлению графиков:** используйте `matplotlib` (без seaborn), один график на фигуру, не задавайте кастомные цвета.

**Зависимости:** `datasets[audio]`, `librosa`, `numpy`, `matplotlib`, `soundfile`, `tqdm`.


In [1]:
# === Установка и импорт (при необходимости раскомментируйте установки) ===
# !pip install -q datasets[audio] librosa soundfile matplotlib numpy tqdm

import json
import os
import random
import warnings

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
from datasets import Audio, load_dataset
from tqdm import tqdm

warnings.filterwarnings("ignore")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)



## Задание 1. Выбор и загрузка датасета(ов)

**Варианты:**
- Речь (например, `PolyAI/minds14`, любая локаль).
- Звуки окружения (любой датасет на HF Hub или локальная папка WAV/FLAC).

**Что сделать:**
1. Указать идентификатор/путь датасета.
2. Загрузить сплит (например, `train` или подвыборку).
3. Показать структуру фич и пример записи.


In [2]:
# Выбор датасеты
DATASET_ID = "PolyAI/minds14"
DATASET_CONFIG = "ru-RU"
DATASET_SPLIT = "train"

# Загзузка датасета
try:
    ds = load_dataset(DATASET_ID, name=DATASET_CONFIG, split=DATASET_SPLIT)
except Exception as e:
    print("Не удалось загрузить датасет через load_dataset:", e)

In [3]:
# Вывод схемы фич и пример
def preview(ds):
    if ds is None:
        print("Dataset is None")
        return
    print(ds)
    print("Пример:")
    print(ds[0])


preview(ds)

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 539
})
Пример:
{'path': 'ru-RU~LATEST_TRANSACTIONS/6030093cbb1e6d0fbce93a74.wav', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x76f5e7c63770>, 'transcription': 'Здравствуйте я бы хотела пересмотреть свои предыдущие последние операции которые проходили по моей карте прямым помимо ему счёту Покажите пожалуйста операции последних трёх месяцев', 'english_transcription': 'Hello, I would like to review my previous last transactions that took place on my card directly in addition to his account. Please show the transactions of the last three months', 'intent_class': 12, 'lang_id': 12}



## Задание 2. Базовые характеристики аудиосигнала

**Что посчитать для N примеров:**
- Частота дискретизации (SR), длительность (с), количество каналов.
- Пиковый уровень (max |x|), RMS‑уровень, оценка динамического диапазона (приближенно).
- Коэффициент клиппинга (% сэмплов с |x|≈1.0 при float или на границе int).
- Zero‑Crossing Rate (ZCR), спектральный центроид и полосa (bandwidth).
- (Опционально) Оценка SNR при простой модели шума (на ваш выбор).

**Вывести:**
- Сводную таблицу (словари/списки), а также агрегаты: mean/median/p95.


In [ ]:
# функции вычисления характеристик
def num_channels(arr: np.ndarray) -> int:
    """Возвращает количество каналов (1 для моно)."""
    if arr.ndim == 1:
        return 1
    if arr.ndim == 2:  # если стерео (C x T или T x C)
        return min(arr.shape)

    return -1


def peak_level(arr: np.ndarray) -> float:
    """Пиковый уровень max |x| для float в [-1, 1]."""
    return float(np.max(np.abs(arr)))


def rms_level(arr: np.ndarray) -> float:
    """RMS-уровень сигнала."""
    return float(np.sqrt(np.mean(arr**2)))


def clipping_ratio(arr: np.ndarray, thr: float = 0.999) -> float:
    """Доля сэмплов близких к клиппингу (|x| >= thr)."""
    total = arr.size
    clipped = np.sum(np.abs(arr) >= thr)
    return float(clipped / total)


def zero_crossing_rate(arr: np.ndarray) -> float:
    """Оценка ZCR (число пересечений нуля / длина)."""
    signs = np.sign(arr)
    not_zero_signs = signs[signs != 0]
    crossings = np.sum(not_zero_signs[1:] != not_zero_signs[:-1])
    return float(crossings / len(arr))


def spectral_stats(arr: np.ndarray, sr: int) -> dict:
    """Спектральные признаки: центроид и полоса (bandwidth)."""
    if arr.ndim > 1:
        arr = librosa.to_mono(arr.T)

    centroid = librosa.feature.spectral_centroid(y=arr, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y=arr, sr=sr)

    return {"centroid": centroid, "bandwidth": bandwidth}


def duration_seconds(arr: np.ndarray, sr: int) -> float:
    return len(arr) / float(sr)


In [ ]:
# TODO-2.2: пройдите по подвыборке датасета и соберите метрики
N = 200  # размер подвыборки для оценки
metrics = []
if ds is not None:
    idxs = np.random.choice(len(ds), size=min(N, len(ds)), replace=False)
    for i in tqdm(idxs, desc="Computing metrics"):
        ex = ds[int(i)]
        aud = ex.get("audio", {})
        arr, sr = aud["array"], aud["sampling_rate"]
        if arr is None or sr is None:
            continue
        if arr.ndim > 1:
            arr = librosa.to_mono(arr.T)  # переводим в моно
        m = {
            "sr": sr,
            "dur_s": duration_seconds(arr, sr),
            "channels": num_channels(arr),
            "peak": peak_level(arr),
            "rms": rms_level(arr),
            "clip_ratio": clipping_ratio(arr),
            "zcr": zero_crossing_rate(arr),
        }
        sc = spectral_stats(
            arr, sr
        )  # ожидается dict с ключами, например, 'centroid', 'bandwidth'
        if isinstance(sc, dict):
            m.update(sc)
        metrics.append(m)

metrics[:3]  # просмотр первых строк


Computing metrics: 100%|██████████| 200/200 [00:02<00:00, 69.82it/s] 


[{'sr': 8000,
  'dur_s': 12.288,
  'channels': 1,
  'peak': 0.8553466796875,
  'rms': 0.11638379096984863,
  'clip_ratio': 0.0,
  'zcr': 0.20734659830729166,
  'centroid': array([[1690.11155807, 1128.80685596, 1019.34058227, 1498.84483201,
          1875.94215287, 2006.53508434, 2034.96089778, 2000.94507249,
          1752.02227744, 1638.5642204 , 1868.29344368, 2837.03130989,
          3033.14621177, 2243.32355169, 2126.94018864, 2046.00293611,
          2046.46612252, 2012.79577931, 1844.00096756, 1289.46755784,
          1166.4959576 , 1104.69742906,  950.59505115,  905.43787346,
          1214.13209489, 1521.17168596,  529.82894079,  495.47978194,
           849.69686435,  979.3444941 , 1085.24760412,  773.75641048,
           631.99616792,  831.43161665,  822.4703022 ,  580.18304811,
           482.00011118,  540.00369086,  718.34134562,  739.80031606,
           817.39906431,  917.36053536, 2034.3288255 , 2569.14790768,
          2271.84300878, 1380.55415141,  711.05002436,  581.

In [ ]:
# TODO-2.3: агрегируйте метрики и выведите сводку (mean/median/p95)
import numpy as np


def agg(values):
    if not values:
        return {"mean": None, "median": None, "p95": None}
    arr = np.array(values, dtype=float)
    return {
        "mean": float(np.mean(arr)),
        "median": float(np.median(arr)),
        "p95": float(np.percentile(arr, 95)),
    }


def summarize(metrics):
    keys = [k for k in metrics[0].keys() if isinstance(metrics[0][k], (int, float))]
    summary = {}
    for k in keys:
        summary[k] = agg([m[k] for m in metrics if m.get(k) is not None])
    return summary


if metrics:
    summary = summarize(metrics)
    print(json.dumps(summary, indent=2, ensure_ascii=False))
else:
    print("Нет метрик для сводки — проверьте загрузку/вычисления.")



## Задание 3. Визуализация: waveform, спектр, Mel/Log‑Mel

**Что сделать:**
1. Построить waveform для нескольких примеров (ось x — время, y — амплитуда).
2. STFT‑спектрограмму в dB (логарифмическая шкала по амплитуде).
3. Mel‑спектрограмму и Log‑Mel при разных параметрах (`n_fft`, `hop_length`, `n_mels`, `fmax`, `htk/slaney`).
4. Сравнить как минимум 3 конфигурации и описать наблюдения в тексте.


In [ ]:
# TODO-3.1: вспомогательные функции визуализации
def plot_waveform(arr, sr, title="Waveform"):
    plt.figure()
    librosa.display.waveshow(arr, sr=sr)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()


def plot_spectrogram_db(arr, sr, n_fft=2048, hop_length=512, title="Spectrogram (dB)"):
    import numpy as np

    D = librosa.stft(arr, n_fft=n_fft, hop_length=hop_length)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    plt.figure()
    librosa.display.specshow(S_db, sr=sr, x_axis="time", y_axis="hz")
    plt.title(title)
    plt.colorbar()
    plt.show()


def plot_mel_logmel(
    arr,
    sr,
    n_fft=2048,
    hop_length=512,
    n_mels=80,
    fmax=None,
    htk=False,
    title_prefix="Mel",
):
    import numpy as np

    mel = librosa.feature.melspectrogram(
        y=arr,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        fmax=fmax,
        htk=htk,
    )
    logmel = librosa.power_to_db(mel, ref=np.max)
    plt.figure()
    librosa.display.specshow(
        logmel, sr=sr, x_axis="time", y_axis="mel", hop_length=hop_length
    )
    plt.title(f"{title_prefix}: log-Mel (n_mels={n_mels}, htk={htk})")
    plt.colorbar()
    plt.show()
    return mel, logmel


In [ ]:
# TODO-3.2: выберите 1-2 примера и постройте все типы визуализаций
if ds is not None and len(ds) > 0:
    ex = ds[0]
    arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
    plot_waveform(arr, sr, title="Waveform — пример 0")
    plot_spectrogram_db(
        arr, sr, n_fft=2048, hop_length=512, title="Spectrogram dB — пример 0"
    )
    # Mel/Log‑Mel: сравнение конфигураций
    cfgs = [
        dict(n_fft=1024, hop_length=256, n_mels=64, fmax=8000, htk=False),
        dict(n_fft=2048, hop_length=512, n_mels=80, fmax=8000, htk=True),
        dict(n_fft=2048, hop_length=256, n_mels=128, fmax=8000, htk=False),
    ]
    for i, cfg in enumerate(cfgs):
        plot_mel_logmel(arr, sr, **cfg, title_prefix=f"Mel cfg #{i + 1}")
else:
    print("Датасет пуст или не загружен.")



## Задание 4. Передискретизация и сравнение (resampling)

**Что сделать:**
1. Передискретизировать сигнал до 16 kHz (если исходно другой SR) и сравнить waveform/спектры до/после.
2. Посчитать Nyquist‑частоту для обеих SR и обсудить, какие частоты потенциально теряются.
3. Оценить влияние `hop_length` при фиксированном `n_fft` на временное/частотное разрешение.


In [ ]:
# TODO-4.1: ресемплинг и сравнение
TARGET_SR = 16_000


def resample(arr, sr, target_sr=TARGET_SR):
    if sr == target_sr:
        return arr, sr
    # TODO: реализуйте через librosa.resample
    raise NotImplementedError


def nyquist(sr):
    return sr / 2.0


if ds is not None and len(ds) > 0:
    ex = ds[0]
    arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
    arr2, sr2 = resample(arr, sr, TARGET_SR)
    # Визуальное сравнение
    plot_waveform(arr, sr, title=f"Waveform SR={sr}")
    plot_waveform(arr2, sr2, title=f"Waveform SR={sr2}")
    plot_spectrogram_db(arr, sr, title=f"Spec dB SR={sr}")
    plot_spectrogram_db(arr2, sr2, title=f"Spec dB SR={sr2}")
    print("Nyquist исходный:", nyquist(sr), "Hz; после:", nyquist(sr2), "Hz")
else:
    print("Датасет пуст или не загружен.")



## Задание 5. Dataset‑level EDA

**Что сделать:**
1. Построить распределения длительностей, SR, пиковых уровней (гистограммы).
2. (Если есть метки классов) — подсчитать число примеров по классам.
3. Сформировать короткие выводы о качестве сырого датасета (наличие очень коротких/длинных записей, клиппинг и пр.).


In [ ]:
# TODO-5.1: гистограммы и распределения
def hist(values, bins=30, title="Histogram", xlabel="value"):
    import matplotlib.pyplot as plt

    plt.figure()
    plt.hist(values, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("count")
    plt.show()


if metrics:
    hist(
        [m["dur_s"] for m in metrics if m.get("dur_s") is not None],
        title="Duration (s)",
        xlabel="seconds",
    )
    hist(
        [m["sr"] for m in metrics if m.get("sr") is not None],
        title="Sampling Rate",
        xlabel="Hz",
    )
    hist(
        [m["peak"] for m in metrics if m.get("peak") is not None],
        title="Peak level",
        xlabel="abs max",
    )
